In [ ]:
import cv2
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import pickle
from keras_facenet import FaceNet



In [ ]:
import time

In [ ]:
from utils import streaming_utils_Tut
streaming_utils_Tut.getStream(0)

In [ ]:
#initialize
facenet = FaceNet()
faces_embeddings = np.load("faces_embeddings_done_4classes.npz")
Y = faces_embeddings['arr_1'] # 0 is for X array, 1 is for Y array
encoder = LabelEncoder()
encoder.fit(Y)
haarcascade = cv2.CascadeClassifier("/Users/intern5/Downloads/haarcascade_frontalface_default.xml") #our face detection model, will probably be mediapipes facenet
model = pickle.load(open("svm_model_160x160.pkl", 'rb'))
#its the image classifier, will use instead of mediaPipes version because this one specializes in faces whereas mediapipes does not. need to download this though
# Start video capture
cap = streaming_utils_Tut.getStream()
if not cap.read()[0]:
    print("[ERROR] Cannot open webcam.")


In [ ]:

while cap.isOpened():
    _, frame = cap.read()
    rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    start_detect = time.time()
    faces = haarcascade.detectMultiScale(gray_img, 1.3, 5) #detects faces in grayscale images and returns a list of rects xywh for each face
    end_detect = time.time()
    detection_latency_ms = (end_detect - start_detect) * 1000
    print(f"Face detection latency: {detection_latency_ms:.2f} ms")

    for x,y,w,h in faces:
        img = rgb_img[y:y+h, x:x+w] # crops to isolate detected face
        img = cv2.resize(img, (160,160)) # 1x160x160x3 # resizes to 160x160
        img = np.expand_dims(img,axis=0) # specifications for image for the neural network
        ypred = facenet.embeddings(img) #passes face to facenet which returns a 512 dim vector
        face_name = model.predict(ypred) # predictor
        final_name = encoder.inverse_transform(face_name)
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,255), 10)
        cv2.putText(frame, str(final_name), (x,y-10), cv2.FONT_HERSHEY_SIMPLEX,
                   1, (0,0,255), 3, cv2.LINE_AA)

    cv2.imshow("Face Recognition:", frame)
    if cv2.waitKey(1) & ord('q') ==27:
        break

cap.release()
cv2.destroyAllWindows

In [ ]:
#currently tensorflow mid to highy 60s
# face detection is in the high 20s 